In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [5]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeConfigurationError: You haven't specified an Api-Key.

In [9]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Parker',
  'subject': 'Psychology',
  'stars': 5,
  'review': 'Dr. Parker is incredibly knowledgeable and passionate about the subject. Her lectures are engaging and she genuinely cares about her students.'},
 {'professor': 'Prof. John Stewart',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Stewart explains complex concepts very clearly, but his grading is quite tough. You really need to put in the work to do well in his class.'},
 {'professor': 'Dr. Alice Mitchell',
  'subject': 'Biology',
  'stars': 3,
  'review': "The material is interesting, but Dr. Mitchell's lectures can be a bit dry at times. Still, she's always willing to help during office hours."},
 {'professor': 'Prof. Michael Hayes',
  'subject': 'History',
  'stars': 5,
  'review': 'Prof. Hayes makes history come alive! His storytelling approach keeps everyone hooked. Exams are fair and directly related to his lectures.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Chemistry',
  

In [13]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [14]:
processed_data[0]

{'values': [0.009336074,
  -0.03415269,
  -0.022316108,
  0.045989275,
  0.0016005147,
  0.01605855,
  0.013997829,
  0.054282423,
  0.007281635,
  -0.01488997,
  0.02578415,
  0.028045919,
  -0.0024156934,
  -0.0042062593,
  0.033247985,
  0.014098351,
  0.010309891,
  -0.034554783,
  0.036540113,
  0.008726654,
  0.040611293,
  -0.012125587,
  0.028397748,
  -0.043953683,
  -0.037947435,
  -0.02608572,
  0.060012233,
  0.037620734,
  -0.014525574,
  0.008173777,
  0.04945732,
  -0.014877405,
  -0.001093973,
  -0.016975822,
  -0.07192421,
  0.04355159,
  -0.013583171,
  0.01823236,
  0.023032334,
  0.020607216,
  0.043652114,
  0.008381105,
  -0.03415269,
  0.041365217,
  0.039882503,
  -0.034052167,
  -0.014776882,
  -0.0015282638,
  0.045461528,
  0.03912858,
  -0.039580934,
  0.0155685,
  0.041641656,
  -0.0062795463,
  -0.033624943,
  0.061369292,
  0.028221833,
  0.023861649,
  0.036364198,
  -0.048678264,
  0.06262583,
  0.0044073053,
  -0.01804388,
  -0.029553764,
  -0.03181553

In [15]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="nsl"
)

{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'nsl': {'vector_count': 20}},
 'total_vector_count': 20}